In [1]:
import cv2
from scipy import optimize as opt
import numpy as np
import sys
import math

In [2]:
np.random.seed(2020)
is_planner = False

In [3]:
image_files = ['chess-1.jpg'] if is_planner==False else ['chess-1.jpg', 'chess-2.jpg', 'chess-3.jpg']
correspondence_points_files = ['correspondence_points-1.txt'] if is_planner==False else ['correspondence_points-1.txt', 'correspondence_points-2.txt', 'correspondence_points-3.txt']
ransac_config_file = 'RANSAC.config'

In [4]:
def load_image(image_file):
    image = cv2.imread(image_file)
    return image;

# Part-1

In [5]:
def feature_extraction(image_files, correspondence_points_files):
    
    ctr = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
    for image_file, correspondence_points_file in zip(image_files, correspondence_points_files):
        
        image = load_image(image_file=image_file)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        world_points = np.zeros((7*7,3), np.float32)
        world_points[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
        
        ret, img_points = cv2.findChessboardCorners(gray_image, (7,7), cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

        if ret:
            img_points_2 = cv2.cornerSubPix(gray_image, img_points, (11,11), (-1,-1), ctr)
            image = cv2.drawChessboardCorners(image, (7,7), img_points_2, ret)
            
            file = open(correspondence_points_file, 'w')
            for wp, ip in zip(world_points, img_points.reshape(-1,2)):
                file.write(f'{str(wp[0])} {str(wp[1])} {str(wp[2])} {str(ip[0])} {str(ip[1])}\n')
            file.close()
        
        cv2.imshow(image_file, image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
feature_extraction(image_files=image_files, correspondence_points_files=correspondence_points_files)
for correspondence_points_file in correspondence_points_files:
    with open(correspondence_points_file, 'r') as file:
        print(f'correspondence points file : {correspondence_points_file}')
        for line in file:
            print(line)
    file.close()

correspondence points file : correspondence_points-1.txt
0.0 0.0 0.0 100.38236 95.73071

1.0 0.0 0.0 150.34842 96.28468

2.0 0.0 0.0 200.33192 96.56557

3.0 0.0 0.0 252.43585 97.379265

4.0 0.0 0.0 303.68777 97.62876

5.0 0.0 0.0 352.5826 97.975815

6.0 0.0 0.0 401.45602 98.53605

0.0 1.0 0.0 99.912506 146.44301

1.0 1.0 0.0 149.79953 146.8645

2.0 1.0 0.0 199.77313 147.50337

3.0 1.0 0.0 251.79451 148.0607

4.0 1.0 0.0 303.30844 148.37553

5.0 1.0 0.0 352.24496 148.46869

6.0 1.0 0.0 401.14093 148.76604

0.0 2.0 0.0 99.844925 196.75513

1.0 2.0 0.0 149.85106 197.39249

2.0 2.0 0.0 199.5298 197.81061

3.0 2.0 0.0 251.13188 198.44028

4.0 2.0 0.0 302.4192 198.54272

5.0 2.0 0.0 351.51013 198.65872

6.0 2.0 0.0 400.56793 198.8078

0.0 3.0 0.0 99.76315 247.1261

1.0 3.0 0.0 149.87688 247.40475

2.0 3.0 0.0 199.35303 247.69678

3.0 3.0 0.0 250.7124 248.15752

4.0 3.0 0.0 301.70706 248.43806

5.0 3.0 0.0 350.496 248.49817

6.0 3.0 0.0 399.75937 248.53786

0.0 4.0 0.0 99.45252 297.33496

1.0

# Part-2

In [7]:
def get_correspondence_data(correspondence_points_files=correspondence_points_files):
    
    world_points = list()
    img_points = list()
    
    for correspondence_points_file in correspondence_points_files:
        wps = list()
        ips = list()
        with open(correspondence_points_file, 'r') as file:
            for line in file:
                points = line.split()
                wps.append([float(points[point]) for point in range(0,3,1)])
                ips.append([float(points[point]) for point in range(3,5,1)])
        
        world_points.append(wps)
        img_points.append(ips)
    
    world_points = np.array(world_points).astype('float32')
    img_points = np.array(img_points).astype('float32')
    
    return world_points, img_points

In [18]:
def get_matirx_A_NP(world_points, img_points):
    A = list()
    for wp,ip in zip(world_points, img_points):
        pi = list()
        xi, yi = ip[0], ip[1]
        eq1, eq2 = list(), list()

        pi.extend(wp)
        pi.append(1)
        
        eq1.extend(pi)
        eq1.extend([0,0,0,0])
        eq1.extend([-1*xi*p for p in pi])
        
        eq2.extend([0,0,0,0])
        eq2.extend(pi)
        eq2.extend([-1*yi*p for p in pi])
        
        eq1, eq2 = np.array(eq1), np.array(eq2)
        
        A.append(eq1)
        A.append(eq2)

    return np.array(A)

def get_matrix_M_NP(A):
    
    U, D, VT = np.linalg.svd(A, full_matrices = True)
    print(D)
    M = VT[-4].reshape(3, 4)
    
    a1 = M[0][:3].T
    a2 = M[1][:3].T
    a3 = M[2][:3].T
    b = M[:,3].reshape((3,1))
    return a1, a2, a3, b, M

def compute_parameters_NP(a1, a2, a3, b):
    rho_abs = 1 / np.linalg.norm(a3.T)
    u0 = (rho_abs ** 2) * (a1.T.dot(a3))
    v0 = (rho_abs ** 2) * (a2.T.dot(a3))
    alpha_v = np.sqrt(((rho_abs ** 2) * a2.T.dot(a2)) - (v0 ** 2))
    s = ((rho_abs ** 4) / alpha_v) * np.cross(a2.T, a3.T).dot(np.cross(a1.T, a3.T).T)
    alpha_u = np.sqrt(((rho_abs ** 2) * a1.T.dot(a1)) - (s ** 2) - (u0 ** 2))
    K_star = np.array([[alpha_u, s, u0],[0, alpha_v, v0],[0, 0, 1]])
    E = np.sign(b[2])
    T_star = E * rho_abs * np.linalg.inv(K_star).dot(b)
    r3 = E * rho_abs * a3
    r1 = ((rho_abs ** 2) / alpha_v) * np.cross(a1, a3)
    r2 = np.cross(r3, r1)
    R_star = np.array([r1.T, r2.T, r3.T]).T
    M = rho_abs * np.array([[a1[0], a1[1], a1[2], b[0][0]],[a2[0], a2[1], a2[2], b[1][0]],[a3[0], a3[1], a3[2], b[2][0]]])
    return u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M

In [19]:
def normalize_points(world_points, img_points):
    views = world_points.shape[0]

    def get_normalization_matrix(points):
        points = points.astype(np.float64)
        
        x_mean, y_mean = np.mean(points, axis=0)
        var_x, var_y = np.var(points, axis=0)
        s_x , s_y = np.sqrt(2/var_x), np.sqrt(2/var_y)

        n = np.array([[s_x, 0, -s_x*x_mean], [0, s_y, -s_y*y_mean], [0, 0, 1]])
        n_inv = np.array([ [1./s_x ,  0 , x_mean], [0, 1./s_y, y_mean] , [0, 0, 1] ])
        
        return n.astype(np.float64), n_inv.astype(np.float64)

    norm_world_points, norm_img_points = list(), list()
    for i in range(views):
        wps, ips = world_points[i], img_points[i]
        
        wps = np.array([np.array([wp[0], wp[1]]) for wp in wps])
        
        N_x, N_x_inv = get_normalization_matrix(wps)
        N_u, N_u_inv = get_normalization_matrix(ips)

        hom_ips = np.array([ [[each[0]], [each[1]], [1.0]] for each in ips])
        hom_wps = np.array([ [[each[0]], [each[1]], [1.0]] for each in wps])

        normalized_hom_ips = hom_ips
        normalized_hom_wps = hom_wps

        for i in range(normalized_hom_wps.shape[0]):
            n_o = np.matmul(N_x,normalized_hom_wps[i])
            normalized_hom_wps[i] = n_o/n_o[-1]
            
            n_u = np.matmul(N_u,normalized_hom_ips[i])
            normalized_hom_ips[i] = n_u/n_u[-1]

        normalized_wps = normalized_hom_wps.reshape(normalized_hom_wps.shape[0], normalized_hom_wps.shape[1])
        normalized_ips = normalized_hom_ips.reshape(normalized_hom_ips.shape[0], normalized_hom_ips.shape[1])

        normalized_wps = normalized_wps[:,:-1]
        normalized_wps = np.array([np.array([nwp[0], nwp[1], 0.0]) for nwp in normalized_wps])
        normalized_ips = normalized_ips[:,:-1]
        
        norm_world_points.append(normalized_wps)
        norm_img_points.append(normalized_ips)
        
    norm_world_points = np.array(norm_world_points)
    norm_img_points = np.array(norm_img_points)
        
    return norm_world_points, norm_img_points, N_x, N_u, N_x_inv, N_u_inv

def get_matirx_A_P(norm_world_points, norm_img_points):
    A = list()
    
    for wps,ips in zip(norm_world_points, norm_img_points):
        a = list()
        
        for wp,ip in zip(wps, ips):
            pi = list()
            xi, yi = ip[0], ip[1]
            eq1, eq2 = list(), list()
            
            pi.extend(wp[:2])
            pi.append(1)
            
            eq1.extend(pi)
            eq1.extend([0, 0, 0])
            eq1.extend([-1*xi*p for p in pi])
            
            eq2.extend([0, 0, 0])
            eq2.extend(pi)
            eq2.extend([-1*yi*p for p in pi])
            
            eq1, eq2 = np.array(eq1), np.array(eq2)
            
            a.append(eq1)
            a.append(eq2)
        
        A.append(np.array(a))
    
    return np.array(A)

def get_matrix_H_P(A, N_wp, N_ip, N_wp_inv, N_ip_inv):
    
    H = list()
    
    for a in A:
        U, D, VT = np.linalg.svd(a)
        h = VT[np.argmin(D)].reshape(3, 3)
        
        h = np.matmul(np.matmul(N_ip_inv,h), N_wp)
        
        H.append(h)
    
    return np.array(H)

def minimizer_func(initial_guess, X, Y, h, N):
    x_j = X.reshape(N, 2)
    projected = [0 for i in range(2*N)]
    for j in range(N):
        x, y = x_j[j]
        w = h[6]*x + h[7]*y + h[8]
        projected[2*j] = (h[0] * x + h[1] * y + h[2]) / w
        projected[2*j + 1] = (h[3] * x + h[4] * y + h[5]) / w

    return (np.abs(projected - Y))**2

def jac_function(initial_guess, X, Y, h, N):
    x_j = X.reshape(N, 2)
    jacobian = np.zeros( (2*N, 9) , np.float64)
    for j in range(N):
        x, y = x_j[j]
        sx = np.float64(h[0]*x + h[1]*y + h[2])
        sy = np.float64(h[3]*x + h[4]*y + h[5])
        w = np.float64(h[6]*x + h[7]*y + h[8])
        jacobian[2*j] = np.array([x/w, y/w, 1/w, 0, 0, 0, -sx*x/w**2, -sx*y/w**2, -sx/w**2])
        jacobian[2*j + 1] = np.array([0, 0, 0, x/w, y/w, 1/w, -sy*x/w**2, -sy*y/w**2, -sy/w**2])

    return jacobian

def get_refine_H_P(H, world_points, img_points):
    
    H_r = list()
        
    for h, wps, ips in zip(H, world_points, img_points):
        N = wps.shape[0]
        wps = np.array([np.array([wp[0], wp[1]]) for wp in wps])
        X = wps.flatten()
        Y = ips.flatten()
        h_u = h.flatten()
        h_prime = opt.least_squares(fun=minimizer_func, x0=h_u, jac=jac_function, method="lm" , args=[X, Y, h_u, N], verbose=0)


        if h_prime.success:
            h =  h_prime.x.reshape(3, 3)
        h = h/h[2, 2]
        
        H_r.append(np.array(h))
    
    return np.array(H_r)

def get_matrix_V_P(H):
    
    V = list()
    
    for h in H:
        V00T = np.array([ h[0, 0]*h[0, 0], h[0, 0]*h[1, 0] + h[1, 0]*h[0, 0], h[1, 0]*h[1, 0], h[2, 0]*h[0, 0] + h[0, 0]*h[2, 0], h[2, 0]*h[1, 0] + h[1, 0]*h[2, 0], h[2, 0]*h[2, 0] ])
        V11T = np.array([ h[0, 1]*h[0, 1], h[0, 1]*h[1, 1] + h[1, 1]*h[0, 1], h[1, 1]*h[1, 1], h[2, 1]*h[0, 1] + h[0, 1]*h[2, 1], h[2, 1]*h[1, 1] + h[1, 1]*h[2, 1], h[2, 1]*h[2, 1] ])
        V01T = np.array([ h[0, 0]*h[0, 1], h[0, 0]*h[1, 1] + h[1, 0]*h[0, 1], h[1, 0]*h[1, 1], h[2, 0]*h[0, 1] + h[0, 0]*h[2, 1], h[2, 0]*h[1, 1] + h[1, 0]*h[2, 1], h[2, 0]*h[2, 1] ])
        
        V1 = V01T
        V2 = np.subtract(V00T , V11T)

        V.append(V1)
        V.append(V2)
    
    return np.array(V)

def compute_parameters_P(H, V):
    
    U, D, VT = np.linalg.svd(V, full_matrices = True)
    S = VT[np.argmin(D)]
        
    c1 = (S[1]*S[3]) - (S[0]*S[4])
    c2 = (S[0]*S[2]) - (S[1]**2)
    v0 = c1 / c2
    lamda = S[5] - (((S[3]**2) + (v0*c1))/S[0])
    alpha_u = np.sqrt(lamda/S[0])
    alpha_v = np.sqrt((lamda*S[0])/c2)
    s = (-1 * S[1] * (alpha_u ** 2) * alpha_v) / lamda
    u0 = ((s*v0)/alpha_u) - ((S[3]*(alpha_u**2))/lamda)
    K_star = np.array([[alpha_u, s, u0],[0, alpha_v, v0],[0, 0, 1]])
    
    T_star = list()
    R_star = list()
    M = list()
    
    for h in H:
        h1 = h[:,0]
        h2 = h[:,1]
        h3 = h[:,2]
        
        alpha_abs = 1 / np.linalg.norm(np.linalg.inv(K_star).dot(h1))
        E = np.sign(np.linalg.inv(K_star).dot(h3)[0])
        alpha = alpha_abs * E
        r1 = alpha * np.linalg.inv(K_star).dot(h1)
        r2 = alpha * np.linalg.inv(K_star).dot(h2)
        r3 = np.cross(r1, r2)
        r_star = np.array([r1, r2, r3])
        t_star = alpha * np.linalg.inv(K_star).dot(h3)
        m = np.matmul(K_star, np.array([[r_star[0][0],r_star[0][1],r_star[0][2],t_star[0]], [r_star[1][0],r_star[1][1],r_star[1][2],t_star[1]], [r_star[2][0],r_star[2][1],r_star[2][2],t_star[2]]]))
        T_star.append(t_star)
        R_star.append(r_star)
        M.append(m)
        
    T_star = np.array(T_star)
    R_star = np.array(R_star)
    M = np.array(M)
    
    H_new = list()
    
    for h in H:
        h1 = h[:,0]
        h2 = h[:,1]
        h3 = h[:,2]
        
        h = np.array([h1.T, h2.T, np.array([0,0,0]), h3.T]).T
        H_new.append(h)
    
    H = np.array(H_new)
    
    return u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M, H

In [20]:
def mean_square_error(M, world_points, img_points):
    m1T = M[0][:4]
    m2T = M[1][:4]
    m3T = M[2][:4]
    mse = 0.0
    for wp, ip in zip(world_points, img_points):
        xi = ip[0]
        yi = ip[1]
        pi = np.array(wp)
        pi = np.concatenate([pi, [1]]).T
        p_xi = (m1T.dot(pi)) / (m3T.dot(pi))
        p_yi = (m2T.dot(pi)) / (m3T.dot(pi))
        mse += (((xi - p_xi) ** 2) + ((yi - p_yi) ** 2))
    mse = mse / len(world_points)
    return mse

In [21]:
world_points, img_points = get_correspondence_data(correspondence_points_files=correspondence_points_files)
print(f'number of points : {world_points.shape} , {img_points.shape}')

number of points : (1, 49, 3) , (1, 49, 2)


In [22]:
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(world_points, img_points, cv2.cvtColor(load_image(image_file=image_files[0]), cv2.COLOR_BGR2GRAY).shape[::-1], None, None)
print(f'matrix :\n {matrix}')
print(f'distortion :\n {distortion}')
print(f'r_vecs :\n {r_vecs}')
print(f't_vecs :\n {t_vecs}')

matrix :
 [[1.72898927e+03 0.00000000e+00 2.48854487e+02]
 [0.00000000e+00 1.71712703e+03 2.54323061e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion :
 [[-4.10528485e+00  4.79951727e+02 -2.15204886e-04  3.16639960e-03
  -1.66044518e+04]]
r_vecs :
 [array([[ 0.05598091],
       [-0.06690154],
       [ 0.0064929 ]])]
t_vecs :
 [array([[-2.9337013],
       [-3.1385576],
       [33.6472396]])]


In [23]:
if is_planner == False:
    
    world_points = world_points[0]
    img_points = img_points[0]
    
    A = get_matirx_A_NP(world_points=world_points, img_points=img_points)
    print(f'shape of A : {A.shape}')
    print()
    
    a1, a2, a3, b, M = get_matrix_M_NP(A=A)
    print(f'a1 :\n {a1}')
    print(f'a2 :\n {a2}')
    print(f'a3 :\n {a3}')
    print(f'b :\n {b}')
    print(f'M :\n {M}')
    print()
    
    u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M = compute_parameters_NP(a1=a1, a2=a2, a3=a3, b=b)
    print(f'u0 : {u0}')
    print(f'v0 : {v0}')
    print(f'alpha_u : {alpha_u}')
    print(f'alpha_v : {alpha_v}')
    print(f's : {s}')
    print(f'K_star :\n {K_star}')
    print(f'T_star :\n {T_star}')
    print(f'R_star :\n {R_star}')
    print(f'M :\n {M}')
    print()
    
    mse = mean_square_error(M=M, world_points=world_points, img_points=img_points)
    print(f'mean square error = {mse}')
    print()

elif is_planner == True:
    
    norm_world_points, norm_img_points, N_wp, N_ip, N_wp_inv, N_ip_inv = normalize_points(world_points=world_points, img_points=img_points)
    print(norm_world_points.shape, norm_img_points.shape)
    
    A = get_matirx_A_P(norm_world_points=norm_world_points, norm_img_points=norm_img_points)
    print(f'shape of A : {A.shape}')
    print()
    
    H = get_matrix_H_P(A=A, N_wp=N_wp, N_ip=N_ip, N_wp_inv=N_wp_inv, N_ip_inv=N_ip_inv)
    print(f'H :\n {H}')
    print()
    
    H_r = get_refine_H_P(H=H, world_points=world_points, img_points=img_points)
    print(f'Refine H :\n {H_r}')
    print()
    
    V = get_matrix_V_P(H=H)
    print(f'V : \n {V}')
    print()
    
    u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M, H = compute_parameters_P(H=H, V=V)
    print(f'u0 : {u0}')
    print(f'v0 : {v0}')
    print(f'alpha_u : {alpha_u}')
    print(f'alpha_v : {alpha_v}')
    print(f's : {s}')
    print(f'K_star :\n {K_star}')
    print(f'T_star :\n {T_star}')
    print(f'R_star :\n {R_star}')
    print(f'M :\n {M}')
    print(f'H :\n {H}')
    print()
    
    mses = list()
    for i in range(H.shape[0]):
        mse = mean_square_error(M=H[i], world_points=world_points[i], img_points=img_points[i])
        mses.append(mse)
    print(f'mean square errors = {mses}')
    print()

shape of A : (98, 12)

[1.48751302e+04 5.45261968e+03 8.54235726e+02 3.26568161e+01
 1.55867299e+01 5.20208205e+00 2.74907956e+00 1.98253532e+00
 3.64900243e-02 2.75759083e-14 3.07423481e-16 1.44415604e-34]
a1 :
 [ 3.27649233e-01 -2.80153091e-04  3.72154564e-14]
a2 :
 [4.27745235e-03 3.25430053e-01 1.18238752e-14]
a3 :
 [1.09710773e-05 9.88341801e-06 0.00000000e+00]
b :
 [[0.63954938]
 [0.61453919]
 [0.00640822]]
M :
 [[ 3.27649233e-01 -2.80153091e-04  3.72154564e-14  6.39549377e-01]
 [ 4.27745235e-03  3.25430053e-01  1.18238752e-14  6.14539190e-01]
 [ 1.09710773e-05  9.88341801e-06  0.00000000e+00  6.40821722e-03]]

u0 : 16473.07511430608
v0 : 14966.02640215756
alpha_u : nan
alpha_v : 16180.230351988674
s : -14865.490053008729
K_star :
 [[            nan -1.48654901e+04  1.64730751e+04]
 [ 0.00000000e+00  1.61802304e+04  1.49660264e+04]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_star :
 [[nan]
 [nan]
 [nan]]
R_star :
 [[-1.04254958e-13  6.14932074e-01  7.42975899e-01]
 [ 1.

C:\Users\yashm\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in sqrt


# Part-3

In [14]:
def get_ransac_config(config_file):
    with open(config_file, 'r') as file:
        P = float(file.readline().split()[0])
        W = float(file.readline().split()[0])
        N = int(file.readline().split()[0])
        D = int(file.readline().split()[0])
        K = int(file.readline().split()[0])
        K_upper = int(file.readline().split()[0])
        T = int(file.readline().split()[0])
    return P, W, N, D, K, K_upper, T

In [15]:
def get_distance_NP(M, world_points, img_points):
    
    m1T = M[0][:4]
    m2T = M[1][:4]
    m3T = M[2][:4]
    
    distances = list()
    
    for wp,ip in zip(world_points, img_points):
        xi = ip[0]
        yi = wp[1]
        pi = np.array(wp)
        pi = np.concatenate([pi, [1]]).T
        p_xi = (m1T.dot(pi)) / (m3T.dot(pi))
        p_yi = (m2T.dot(pi)) / (m3T.dot(pi))
        distance = np.sqrt((((xi - p_xi) ** 2) + ((yi - p_yi) ** 2)))
        distances.append(distance)
    
    return distances

In [16]:
def ransac(world_points, img_points, P, W, N, D, K, K_upper, T):
    
    best_model = None
    current_K = 0
    current_inliners = D
    A = get_matirx_A_NP(world_points=world_points, img_points=img_points)
    a1, a2, a3, b, M = get_matrix_M_NP(A=A)
    distances = get_distance_NP(M=M, world_points=world_points, img_points=img_points)
    median_distance = np.median(distances)
    T = 1.5 * median_distance
    
    while(current_K < K):
        
        selected_indices = np.random.choice(world_points.shape[0], N)
        random_world_points, random_img_points = world_points[selected_indices], img_points[selected_indices]
                
        A = get_matirx_A_NP(world_points=random_world_points, img_points=random_img_points)
        a1, a2, a3, b, M = get_matrix_M_NP(A=A)
        
        distances = get_distance_NP(M=M, world_points=world_points, img_points=img_points)
        
        inliners = list()
        
        for index, distance in zip(range(len(distances)), distances):
            if distance < T:
                inliners.append(index)
                
        if len(inliners) >= current_inliners:
            current_inliners = len(inliners)
            inliner_world_points, inliner_img_points = world_points[inliners], img_points[inliners]
            A = get_matirx_A_NP(world_points=inliner_world_points, img_points=inliner_img_points)
            a1, a2, a3, b, M = get_matrix_M_NP(A=A)
            distances = get_distance_NP(M=M, world_points=world_points, img_points=img_points)
            best_model = M
        
        median_distance = np.median(distances)
        T = 1.5 * median_distance
#         if W != 0:
        W = len(inliners)/world_points.shape[0]
        K = np.absolute(math.log(1 - P) / math.log(1 - (W ** N)))
        K = K_upper if K >= K_upper else K
        current_K += 1;
    
    return best_model, K, W, T

In [17]:
world_points, img_points = get_correspondence_data(correspondence_points_files=correspondence_points_files)
world_points = world_points[0]
img_points = img_points[0]
print(f'number of points : {world_points.shape} , {img_points.shape}')
print()

P, W, N, D, K, K_upper, T = get_ransac_config(config_file=ransac_config_file)
print(f'P : {P}')
print(f'W : {W}')
print(f'N : {N}')
print(f'D : {D}')
print(f'K : {K}')
print(f'K_upper : {K_upper}')
print(f'T : {T}')
print()

best_model, K, W, T = ransac(world_points=world_points, img_points=img_points, P=P, W=W, N=N, D=D, K=K, K_upper=K_upper, T=T)
print(f'Best Model :\n {best_model}')
print(f'K : {K}')
print(f'W : {W}')
print(f'T : {T}')
print()

a1 = best_model[0][:3].T
a2 = best_model[1][:3].T
a3 = best_model[2][:3].T
b = best_model[:,3].reshape((3,1))
u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M = compute_parameters_NP(a1=a1, a2=a2, a3=a3, b=b)
print(f'u0 : {u0}')
print(f'v0 : {v0}')
print(f'alpha_u : {alpha_u}')
print(f'alpha_v : {alpha_v}')
print(f's : {s}')
print(f'K_star :\n {K_star}')
print(f'T_star :\n {T_star}')
print(f'R_star :\n {R_star}')
print(f'M :\n {M}')
print()

mse = mean_square_error(M=M, world_points=world_points, img_points=img_points)
print(f'mean square error = {mse}')
print()

number of points : (49, 3) , (49, 2)

P : 0.99
W : 0.5
N : 12
D : 6
K : 500
K_upper : 1000
T : 100

Best Model :
 [[-3.28260694e-01 -3.81706651e-04 -5.85855149e-14 -6.38961716e-01]
 [-4.38396566e-03 -3.26660259e-01  1.27675648e-15 -6.14170555e-01]
 [-1.15661818e-05 -1.33878043e-05  0.00000000e+00 -6.40952566e-03]]
K : 26.914215878094872
W : 0.8571428571428571
T : 367.3092295817874

u0 : 12146.048724908771
v0 : 14133.641980458327
alpha_u : 0.000244140625
alpha_v : 11883.076789001925
s : -14025.995781897922
K_star :
 [[ 2.44140625e-04 -1.40259958e+04  1.21460487e+04]
 [ 0.00000000e+00  1.18830768e+04  1.41336420e+04]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_star :
 [[-4.24630094e+10]
 [-4.27974269e+02]
 [ 3.62282102e+02]]
R_star :
 [[-2.10868971e-13  8.93172089e-01  6.53748945e-01]
 [ 1.82176912e-13 -7.71641899e-01  7.56711515e-01]
 [ 1.18033368e+00  2.78664942e-13 -0.00000000e+00]]
M :
 [[-1.85540991e+04 -2.15749955e+01 -3.31139691e-09 -3.61156824e+04]
 [-2.47792485e+02 -1.